In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import MinMaxScaler

from mlxtend.feature_selection import SequentialFeatureSelector

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Lasso, ElasticNet, Ridge

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold


import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense      # create layers

from scipy.stats import skew

from xgboost import XGBRegressor

from time import time

# Set de entrenamiento y Test final

Cargamos los archivos necesarios

In [2]:
train = pd.read_csv('train_final.csv')
test = pd.read_csv('test_final.csv')

In [3]:
train.person.count()

19414

In [4]:
test.person.count()

19415

Nos guardamos los IDs necesarios para el submit

In [5]:
id_personas = test['person']

Analizamos el set de entrenamiento y el test final

In [6]:
train.sample(5)

,person,label,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,cant_sesiones_q_0,cant_sesiones_q_1,cant_sesiones_q_2,cant_sesiones_q_3,cant_sesiones_q_4,cant_sesiones_q_5,cant_sesiones_q_6,cant_sesiones_q_7,cant_sesiones_q_8,cant_sesiones_q_9
5413,925d7709,0,3.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
2043,b4ebb2a0,0,6.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
3036,3300a044,0,3.0,1.0,4.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5408,ce2f5d19,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
16257,8575b73a,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Columns: 159 entries, person to cant_sesiones_q_9
dtypes: float64(156), int64(2), object(1)
memory usage: 23.6+ MB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19415 entries, 0 to 19414
Columns: 158 entries, person to cant_sesiones_q_9
dtypes: float64(156), int64(1), object(1)
memory usage: 23.4+ MB


Normalización de las columnas entre 0 y 1

In [9]:

train.iloc[:,2:] = MinMaxScaler().fit_transform(train.iloc[:,2:])

Skew de las columnas, mientras más distinto a 0, menos se parece a una distribución normal

In [10]:
for i in train.iloc[:,2:].columns:
    print(i+':', round(skew(train[i]),2) ) 

ad campaign hit: 15.05
brand listing: 42.74
checkout: 7.1
conversion: 10.96
generic listing: 11.87
lead: 37.0
search engine hit: 10.48
searched products: 18.42
staticpage: 16.57
viewed product: 9.46
visited site: 7.37
viewed product_0: 44.27
viewed product_1: 29.59
viewed product_2: 39.5
viewed product_3: 25.64
viewed product_4: 18.08
viewed product_5: 24.11
viewed product_6: 21.98
viewed product_7: 19.77
viewed product_8: 10.46
viewed product_9: 7.26
searched products_0: 43.21
searched products_1: 25.47
searched products_2: 28.54
searched products_3: 33.04
searched products_4: 24.93
searched products_5: 35.93
searched products_6: 30.65
searched products_7: 16.97
searched products_8: 22.44
searched products_9: 21.84
ad campaign hit_0: 28.67
ad campaign hit_1: 31.3
ad campaign hit_2: 56.25
ad campaign hit_3: 64.93
ad campaign hit_4: 31.0
ad campaign hit_5: 21.04
ad campaign hit_6: 23.88
ad campaign hit_7: 27.8
ad campaign hit_8: 11.12
ad campaign hit_9: 6.15
staticpage_0: 44.82
staticpa

Se calcula el logaritmo natural de cada valor de la columna + 1.  (De las columnas que tienen skew mayor a 0.5)

In [11]:
skewed_features = np.array(train.iloc[:,2:].columns)[np.abs(skew(train[train.iloc[:,2:].columns])) > 0.5]
train[skewed_features] = np.log1p(train[skewed_features])

In [12]:
skewed_features = np.array(test.iloc[:,1:].columns)[np.abs(skew(test[test.iloc[:,1:].columns])) > 0.5]
test[skewed_features] = np.log1p(test[skewed_features])

Nos guardamos los features que utilizaremos para los algoritmos de Machine Learning

In [67]:
cantidad_sesiones = []
promedio_tiempo_sesion = []
maxima_sesion = []
compras = []
ad_campaign_hit = []
viewed_product = []
brand_listing = []
checkout = []
generic_listing = []
search_engine_hit = []
searched_products = []
viewed_products = []
visited_site = []
lead = []
for i in range(4,10):
    promedio_tiempo_sesion.append('prom_tiempo_sesion_q_'+str(i))
    maxima_sesion.append('max_tiempo_sesion_q_'+str(i))
    cantidad_sesiones.append('cant_sesiones_q_'+str(i))    
for i in range(4,10):
    compras.append('conversion_'+str(i))
    ad_campaign_hit.append('ad campaign hit_'+str(i))
    viewed_product.append('viewed product_'+str(i))
    visited_site.append('visited site_'+str(i))
    brand_listing.append('brand listing_'+str(i))
    checkout.append('checkout_'+str(i))
    generic_listing.append('generic listing_'+str(i))
    search_engine_hit.append('search engine hit_'+str(i))
    searched_products.append('searched products_'+str(i))
    lead.append('lead_'+str(i))
conversiones_dias_features = ['registration_time','max_sin_convertir','tiempo_sin_convertir',\
                              'convierte_cada_x_tiempo']

celular_viejo = ['celularViejo']

pais = ['country']

eventos_total = ['ad campaign hit','viewed product','brand listing','checkout','generic listing',\
                 'search engine hit','searched products','conversion'] 
eventos_out = ['lead','staticpage','visited site']

features_nn = maxima_sesion + \
    conversiones_dias_features +  \
    eventos_total + compras
#numero de las columnas para usar en el nn
num_col = []
for i in features_nn:
    num_col.append(train.columns.get_loc(i))
    
features = eventos_total + conversiones_dias_features + celular_viejo + promedio_tiempo_sesion + maxima_sesion\
            + compras + pais  \
            + ad_campaign_hit + viewed_product + visited_site + lead + checkout\
            + searched_products + search_engine_hit + brand_listing + generic_listing + cantidad_sesiones

In [35]:
#TODOS LOS FEATURES
features = train.iloc[:,2:].columns

Particionamos el set de entrenamiento para realizar pruebas locales de hiper-parámetros antes de realizar cada submit

In [68]:
test_s = 0.25
random_s = 0

x = np.array(train[features])
y = np.array(train['label'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

Generamos el set de entrenamiento para el test final

In [69]:
x_test_final = np.array(test[features])

# Algoritmos probados

### Feature importance

In [21]:
xgb = XGBRegressor()
xgb.fit(x_train, y_train)
imp = pd.DataFrame(xgb.feature_importances_ ,columns = ['Importance'],index = features)
imp = imp.sort_values(['Importance'], ascending = False)
print(imp)

                       Importance
checkout_9               0.079447
conversion               0.046632
dias_sin_convertir       0.041451
max_tiempo_sesion_q_0    0.034542
checkout                 0.031088
max_sin_convertir        0.029361
viewed product           0.029361
max_tiempo_sesion_q_8    0.025907
convierte_cada_x_dias    0.025907
max_tiempo_sesion_q_9    0.024180
viewed product_9         0.022453
ad campaign hit          0.022453
searched products        0.017271
cant_sesiones_q_1        0.015544
max_tiempo_sesion_q_6    0.015544
checkout_8               0.015544
searched products_9      0.015544
brand listing_1          0.013817
search engine hit        0.013817
viewed product_5         0.013817
generic listing_7        0.013817
search engine hit_9      0.013817
brand listing_8          0.013817
ad campaign hit_9        0.013817
checkout_3               0.013817
checkout_7               0.013817
brand listing_7          0.012090
brand listing            0.012090
visited site_4

### Feature Selection

In [14]:
len(features)

147

Selector nos elige los mejores features, entre lo que se pida.

In [15]:

xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)
selector = SequentialFeatureSelector(xgb, scoring='roc_auc', \
                                      verbose=2, k_features=140, forward=False, n_jobs=-1)

selector.fit(x, y)


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

SequentialFeatureSelector(clone_estimator=True, cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0),
             floating=False, forward=False, k_features=140, n_jobs=-1,
             pre_dispatch='2*n_jobs', scoring='roc_auc', verbose=2)

In [16]:
xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)
x_train2 = selector.transform(x_train)  
x_test2 = selector.transform(x_test)
xgb.fit(x_train2,y_train)
y_pred = xgb.predict(x_test2)
roc_auc_score(y_test,y_pred)

0.8614258235804075

Función que remueve los features si el score es mayor.

In [34]:
n_feat = 0  # initialize the number of features we choose, which will be updated in the loop

features = train.iloc[:,2:].columns
feats = []

# start from the top 10 features, then add more less important ones
xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)
xgb.fit(x_train,y_train)
y_pred = xgb.predict(x_test)
best_score = roc_auc_score(y_test,y_pred)
feats = features
feats = list(feats)
for i in features:
    feats.remove(i)
    xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)
    num_col=[]
    for j in feats:
        num_col.append(train.drop(columns=['person','label']).columns.get_loc(j))
    xgb.fit(x_train[:,num_col],y_train)
    y_pred = xgb.predict(x_test[:,num_col])
    score = roc_auc_score(y_test,y_pred)
# remove  feature only if score improves by more than 0.001%
    if (score - best_score) > 10e-3:
        best_score = score
        continue
    feats.append(i)

In [35]:
best_score

0.8621195201775456

In [ ]:
person = np.array(test['person'])
person =person.reshape((19415,1))
submit = pd.DataFrame(data=np.column_stack((person,pred_final)),columns=['person','label'])
submit.to_csv('submit_nn.csv', index=False)

### Majority Voting


In [28]:

params_optimos_gb = {'criterion': 'friedman_mse','learning_rate':0.2, 'n_estimators':10, 'max_depth':5,\
                     'min_samples_split':0.5,'min_samples_leaf':0.1, 'subsample':0.9, 'max_features':'sqrt',\
                     } #'loss': 'deviance'

# Creamos el gradient boosting
gra_boost = GradientBoostingRegressor(**params_optimos_gb)

naive_bayes = GaussianNB()

# Initialising the NN
nn = Sequential()

# layers
nn.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = len(features_nn)))
nn.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
nn.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
nn.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)
ridge = KernelRidge(alpha=0.15, coef0=3.7, degree=2, kernel='polynomial')

ridge.fit(x_train,y_train)
gra_boost.fit(x_train,y_train)
naive_bayes.fit(x_train,y_train)
xgb.fit(x_train,y_train)
nn.fit(x_train[:,num_col], y_train, batch_size = 32, epochs = 200)


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f1f6b459cf8>>
Traceback (most recent call last):
  File "/home/mariano/ENTER/lib/python3.6/site-packages/xgboost-0.72-py3.6.egg/xgboost/core.py", line 453, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
Using TensorFlow backend.


Epoch 1/200
14560/14560 [==============================] - 2s 109us/step - loss: 0.3607 - acc: 0.9489
Epoch 2/200
14560/14560 [==============================] - 0s 32us/step - loss: 0.2057 - acc: 0.9492
Epoch 3/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.2002 - acc: 0.9492
Epoch 4/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1965 - acc: 0.9492
Epoch 5/200
14560/14560 [==============================] - 0s 30us/step - loss: 0.1948 - acc: 0.9492
Epoch 6/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1937 - acc: 0.9492
Epoch 7/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1931 - acc: 0.9492
Epoch 8/200
14560/14560 [==============================] - 0s 30us/step - loss: 0.1927 - acc: 0.9492
Epoch 9/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1921 - acc: 0.9492
Epoch 10/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1918 - 

14560/14560 [==============================] - 0s 32us/step - loss: 0.1849 - acc: 0.9492
Epoch 82/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1849 - acc: 0.9492
Epoch 83/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1849 - acc: 0.9492
Epoch 84/200
14560/14560 [==============================] - 0s 30us/step - loss: 0.1848 - acc: 0.9492
Epoch 85/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1845 - acc: 0.9492
Epoch 86/200
14560/14560 [==============================] - 0s 34us/step - loss: 0.1845 - acc: 0.9492
Epoch 87/200
14560/14560 [==============================] - 0s 32us/step - loss: 0.1846 - acc: 0.9492
Epoch 88/200
14560/14560 [==============================] - 0s 32us/step - loss: 0.1844 - acc: 0.9492
Epoch 89/200
14560/14560 [==============================] - 1s 35us/step - loss: 0.1844 - acc: 0.9492
Epoch 90/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1843 - acc: 

14560/14560 [==============================] - 0s 31us/step - loss: 0.1813 - acc: 0.9492
Epoch 161/200
14560/14560 [==============================] - 0s 32us/step - loss: 0.1812 - acc: 0.9492
Epoch 162/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1813 - acc: 0.9492
Epoch 163/200
14560/14560 [==============================] - 0s 30us/step - loss: 0.1814 - acc: 0.9492
Epoch 164/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1810 - acc: 0.9492
Epoch 165/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1809 - acc: 0.9492
Epoch 166/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1811 - acc: 0.9492
Epoch 167/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.1813 - acc: 0.9492
Epoch 168/200
14560/14560 [==============================] - 0s 32us/step - loss: 0.1809 - acc: 0.9492
Epoch 169/200
14560/14560 [==============================] - 0s 31us/step - loss: 0.181

In [33]:
Y_pred = pd.DataFrame({'person' : y_test})
Y_pred['gra_boost'] = gra_boost.predict(x_test)
Y_pred['xgb'] = xgb.predict(x_test)
Y_pred['nn'] = nn.predict(x_test[:,num_col])

In [34]:
Y_pred['label'] = 0
Y_pred['label'] = Y_pred.iloc[:, 1:3].mode(axis=1)

In [35]:
roc_auc_score(y_test,Y_pred['label'])

0.8539521926022252

## Neural Network

### Set de entrenamiento

In [27]:
# Initialising the NN
model = Sequential()

# layers
model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = len(features)))
model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN
model.fit(x_train, y_train, batch_size = 32, epochs = 200)

Epoch 1/200
7836/7836 [==============================] - 1s 66us/step - loss: 0.3869 - acc: 0.9551
Epoch 2/200
7836/7836 [==============================] - 0s 33us/step - loss: 0.1782 - acc: 0.9587
Epoch 3/200
7836/7836 [==============================] - 0s 32us/step - loss: 0.1642 - acc: 0.9587
Epoch 4/200
7836/7836 [==============================] - 0s 34us/step - loss: 0.1592 - acc: 0.9587
Epoch 5/200
7836/7836 [==============================] - 0s 32us/step - loss: 0.1557 - acc: 0.9587
Epoch 6/200
7836/7836 [==============================] - 0s 32us/step - loss: 0.1537 - acc: 0.9587
Epoch 7/200
7836/7836 [==============================] - 0s 33us/step - loss: 0.1523 - acc: 0.9587
Epoch 8/200
7836/7836 [==============================] - 0s 32us/step - loss: 0.1504 - acc: 0.9587
Epoch 9/200
7836/7836 [==============================] - 0s 34us/step - loss: 0.1493 - acc: 0.9587
Epoch 10/200
7836/7836 [==============================] - 0s 34us/step - loss: 0.1485 - acc: 0.9587
Epoch 11/

7836/7836 [==============================] - 0s 34us/step - loss: 0.1362 - acc: 0.9587
Epoch 84/200
7836/7836 [==============================] - 0s 36us/step - loss: 0.1359 - acc: 0.9587
Epoch 85/200
7836/7836 [==============================] - 0s 35us/step - loss: 0.1360 - acc: 0.9587
Epoch 86/200
7836/7836 [==============================] - 0s 36us/step - loss: 0.1358 - acc: 0.9587
Epoch 87/200
7836/7836 [==============================] - 0s 34us/step - loss: 0.1355 - acc: 0.9587
Epoch 88/200
7836/7836 [==============================] - 0s 34us/step - loss: 0.1356 - acc: 0.9587
Epoch 89/200
7836/7836 [==============================] - 0s 36us/step - loss: 0.1358 - acc: 0.9587
Epoch 90/200
7836/7836 [==============================] - 0s 34us/step - loss: 0.1356 - acc: 0.9587
Epoch 91/200
7836/7836 [==============================] - 0s 32us/step - loss: 0.1355 - acc: 0.9587
Epoch 92/200
7836/7836 [==============================] - 0s 34us/step - loss: 0.1353 - acc: 0.9587
Epoch 93/200


In [28]:
y_pred = model.predict(x_test)
precision = roc_auc_score(y_test,y_pred)
precision

0.7960543873695448

### Prediccion del test final

In [240]:
# Initialising the NN
model = Sequential()

# layers
model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = len(features)))
model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the ANN
model.fit(x, y, batch_size = 32, epochs = 200)

Epoch 1/200
19414/19414 [==============================] - 3s 143us/step - loss: 0.2797 - acc: 0.9492
Epoch 2/200
19414/19414 [==============================] - 1s 64us/step - loss: 0.1869 - acc: 0.9495
Epoch 3/200
19414/19414 [==============================] - 1s 65us/step - loss: 0.1801 - acc: 0.9495
Epoch 4/200
19414/19414 [==============================] - 1s 67us/step - loss: 0.1775 - acc: 0.9495
Epoch 5/200
19414/19414 [==============================] - 1s 66us/step - loss: 0.1754 - acc: 0.9495
Epoch 6/200
19414/19414 [==============================] - 1s 67us/step - loss: 0.1740 - acc: 0.9495
Epoch 7/200
19414/19414 [==============================] - 1s 67us/step - loss: 0.1734 - acc: 0.9495
Epoch 8/200
19414/19414 [==============================] - 1s 64us/step - loss: 0.1724 - acc: 0.9495
Epoch 9/200
19414/19414 [==============================] - 1s 69us/step - loss: 0.1719 - acc: 0.9495
Epoch 10/200
19414/19414 [==============================] - 1s 69us/step - loss: 0.1713 - 

19414/19414 [==============================] - 1s 62us/step - loss: 0.1542 - acc: 0.9495
Epoch 161/200
19414/19414 [==============================] - 1s 62us/step - loss: 0.1540 - acc: 0.9495
Epoch 162/200
19414/19414 [==============================] - 1s 60us/step - loss: 0.1544 - acc: 0.9495
Epoch 163/200
19414/19414 [==============================] - 1s 61us/step - loss: 0.1545 - acc: 0.9495
Epoch 164/200
19414/19414 [==============================] - 1s 61us/step - loss: 0.1537 - acc: 0.9495
Epoch 165/200
19414/19414 [==============================] - 1s 61us/step - loss: 0.1540 - acc: 0.9495
Epoch 166/200
19414/19414 [==============================] - 1s 63us/step - loss: 0.1540 - acc: 0.9495
Epoch 167/200
19414/19414 [==============================] - 1s 64us/step - loss: 0.1541 - acc: 0.9495
Epoch 168/200
19414/19414 [==============================] - 1s 61us/step - loss: 0.1545 - acc: 0.9495
Epoch 169/200
19414/19414 [==============================] - 1s 60us/step - loss: 0.154

In [248]:
pred_final = model.predict_proba(x_test_final)


In [249]:
person = np.array(test['person'])

In [250]:
person =person.reshape((19415,1))

In [251]:
submit = pd.DataFrame(data=np.column_stack((person,pred_final)),columns=['person','label'])

In [252]:
submit.to_csv('submit_nn.csv', index=False)

## XGBoost

### Grid Search

In [42]:


t0 = time()
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

params = {
        'min_child_weight': [5],
        'gamma': [5],
        'subsample': [1.0],
        'colsample_bytree': [0.6],
        'max_depth': [5]
        }


# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate=0.02,silent=True, \
        nthread=1,base_score=0.5, booster='gbtree',\
       colsample_bytree=0.6, gamma=5,max_delta_step=0,random_state=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, objective='binary:logistic',subsample=1.0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1,seed=None)
# ,
#s
#,
folds = 20
param_comb = 1

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgboost,param_distributions=params,\
                                   n_iter=1, scoring='roc_auc',\
                                   n_jobs=4, cv=skf.split(x,y), verbose=3,\
                                   random_state=1001 )

random_search.fit(x,y)

print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Fitting 20 folds for each of 1 candidates, totalling 20 fits
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6, score=0.8046836956040551, total=  32.8s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6, score=0.8560693257780336, total=  39.2s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=0.6, score=0.8575523484882022, total=  43.2s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5

[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  3.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:  3.3min finished



 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0)

 Best normalized gini score for 20-fold search with 1 parameter combinations:
0.7104154374997476

 Best hyperparameters:
{'subsample': 1.0, 'min_child_weight': 5, 'max_depth': 5, 'gamma': 5, 'colsample_bytree': 0.6}
Tiempo de ejecución: 223.66826 seconds.


### Set de entrenamiento

In [70]:
xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)


In [71]:
xgb.fit(x_train,y_train)
pred = xgb.predict(x_test)
roc_auc_score(y_test,pred)

0.8607602766941194

### Prediccion del test final

In [44]:
xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)
xgb.fit(x,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0)

In [45]:
pred_final = xgb.predict(x_test_final)
person = np.array(test['person'])
person =person.reshape((19415,1))
submit = pd.DataFrame(data=np.column_stack((person,pred_final)),columns=['person','label'])
submit.to_csv('submit_xgb.csv', index=False)

# KNN

### Valores iniciales para K

In [75]:
t0 = time()

print('VALORES INICIALES PARA K')

k_valores = [3,5,10,20,50,60,75,80,85,90,95,100]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = roc_auc_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES INICIALES PARA K
La precisión para k= 3 es: 0.5672626788036411
La precisión para k= 5 es: 0.6030983600635748
La precisión para k= 10 es: 0.6973992197659297
La precisión para k= 20 es: 0.738579594711747
La precisión para k= 50 es: 0.7482715648027741
La precisión para k= 60 es: 0.7509987718537783
La precisión para k= 75 es: 0.7549180031787314
La precisión para k= 80 es: 0.7550624909695132
La precisión para k= 85 es: 0.753958965467418
La precisión para k= 90 es: 0.7551907238838318
La precisión para k= 95 es: 0.7568387877474354
La precisión para k= 100 es: 0.7568103417136253
Con k= 95 se obtuvo la mayor precisión: 0.7568387877474354
Tiempo de ejecución: 139.07555 seconds.


### Valores grid search para K

In [76]:
t0 = time()

print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = roc_auc_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES GRID SEARCH PARA K
La precisión para k = 93 es: 0.7565796127727207
La precisión para k = 94 es: 0.7566884301401532
La precisión para k = 95 es: 0.7568387877474354
La precisión para k = 96 es: 0.7580118479988441
La precisión para k = 97 es: 0.7578506538072534
Con k= 96 se obtuvo la mayor precisión: 0.7580118479988441
Tiempo de ejecución: 57.16458 seconds.


In [ ]:
if precision >= mejor_precision:
    mejor_precision = precision
    mejores_features = features

### Predecimos el test final con el mejor k obtenido

In [57]:
# Creamos el KNN
knn = KNeighborsClassifier(n_neighbors=1)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test_final)

### Submit File

In [60]:
person = test_final['person']

submit = pd.DataFrame({'person':person, 'label':pred_final})

submit.to_csv('submit_knn.csv', index=False)

# Random Forest

### Random Search

In [40]:
t0 = time()

params = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = params, n_iter = 100, cv = 3, \
                                                               verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x, y)


tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=True 
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=True 
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=True 
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=60, bootstrap=True 
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=60, bootstrap=True 
[CV] n_estimators=1200, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=60, bootstrap=True 
[CV] n_estimators=1400, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True 
[CV] n_estimators=1400, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True 
[CV]  n_estimators

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 24.7min


[CV]  n_estimators=1400, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=26.3min
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=1400, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=None, bootstrap=True, total=24.8min
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=True, total=30.7min
[CV] n_estimators=2000, min_samples_split=2, min_samples_leaf=4, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=1800, min_samples_split=5, min_samples_leaf=2, max_features=auto, max_depth=30, bootstrap=True, total=31.0min
[CV] n_estimators=2000, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=90, bootstrap=False 
[CV]  n_estimators=

KeyboardInterrupt: 

### Set de entrenamiento

In [74]:
t0 = time()

# Parámetros óptimos
params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el Random Forest
random_forest = RandomForestRegressor(**params_optimos_rf)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.8525362122525647
Tiempo de ejecución: 0.54864 seconds.


### Test final

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = random_forest.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit files

#### Submit con probabilidades

#  Extra Trees

### Set de entrenamiento

In [72]:
t0 = time()

# Parámetros óptimos
params_optimos_et = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el extra_trees
extra_trees = ExtraTreesRegressor(**params_optimos_et)

# Lo entrenamos con nuestro set de entrenamiento
extra_trees.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = extra_trees.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.7832972113856379
Tiempo de ejecución: 0.62484 seconds.


### Test final

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesRegressor(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = extra_trees.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con probabilidades

# Gaussian Naive Bayes

### Set de entrenamiento

In [24]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con nuestro set de entrenamiento
naive_bayes.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = naive_bayes.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.6684005201560468
Tiempo de ejecución: 0.04156 seconds.


### Test final

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = naive_bayes.predict_proba(test)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con probabilidades

#  Perceptron

### Set de entrenamiento

In [ ]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con nuestro set de entrenamiento
perceptron.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = perceptron.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con la totalidad del set de datos
perceptron.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = perceptron.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

# Adaptive Boosting

### Set de entrenamiento

In [55]:
t0 = time()

# Parámetros óptimos
params_optimos_ab = {'n_estimators':50}

# Creamos el adaptive boosting
ada_boost = AdaBoostRegressor(**params_optimos_ab)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.8205037205606126
Tiempo de ejecución: 1.04821 seconds.


### Test final

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostRegressor(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = ada_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con probabilidades

# Gradient Boosting

### Grid Search

129.3min

In [58]:

#parameters = {
#    "loss":["deviance"],
#   "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#    "min_samples_split": [0.1,0.5],#np.linspace(0.1, 0.5, 12)
#    "min_samples_leaf": [0.1,0.5],#np.linspace(0.1, 0.5, 12)
#    "max_depth":[3,5,8],
#    "max_features":["log2","sqrt"],
#    "criterion": ["friedman_mse",  "mae"],
#    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
#    "n_estimators":[10]
#    }
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[5],
    "max_features":["sqrt"],
    "criterion": ["friedman_mse"],
    "subsample":[0.9],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=3, n_jobs=-1,verbose=3,scoring='roc_auc')

clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))
print(clf.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.1, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.13636363636363638, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.13636363636363638, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.

[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.3545454545454546, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.390909090909091, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.390909090909091, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.31818181818181823, n_estimators=10, subsample=0.9, score=0.8186380696862458, total=   0.2s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.390909090909091, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_m

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.9s


[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.42727272727272736, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.3545454545454546, n_estimators=10, subsample=0.9, score=0.8193947074026586, total=   0.2s
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.31818181818181823, n_estimators=10, subsample=0.9, score=0.8310214848783312, total=   0.3s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.42727272727272736, n_estimators=10, subsample=0.9 
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.1, min_samples_split=0.42727272727272736, n_estimator

[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.13636363636363638, min_samples_split=0.2090909090909091, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.13636363636363638, min_samples_split=0.13636363636363638, n_estimators=10, subsample=0.9, score=0.8097535083471815, total=   0.3s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.13636363636363638, min_samples_split=0.24545454545454548, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.13636363636363638, min_samples_split=0.17272727272727273, n_estimators=10, subsample=0.9, score=0.7757131596083763, total=   0.2s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samp

...

[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374, min_samples_split=0.31818181818181823, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374, min_samples_split=0.24545454545454548, n_estimators=10, subsample=0.9, score=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374, min_samples_split=0.31818181818181823, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374, min_samples_split=0.28181818181818186, n_estimators=10, subsample=0.9, score=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374,

[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.5, min_samples_split=0.1, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374, min_samples_split=0.5, n_estimators=10, subsample=0.9, score=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.5, min_samples_split=0.13636363636363638, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374, min_samples_split=0.5, n_estimators=10, subsample=0.9, score=0.5, total=   0.1s
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.46363636363636374, min_samples_split=0.5, n_estimators=10, subsample=0.9, score=0.5, total=   0.1

[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.5, min_samples_split=0.3545454545454546, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.5, min_samples_split=0.31818181818181823, n_estimators=10, subsample=0.9, score=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.5, min_samples_split=0.390909090909091, n_estimators=10, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.5, min_samples_split=0.31818181818181823, n_estimators=10, subsample=0.9, score=0.5, total=   0.1s
[CV] criterion=friedman_mse, learning_rate=0.2, loss=deviance, max_depth=5, max_features=sqrt, min_samples_leaf=0.5, min_samples_split=0.390909090909091, n_estimators=10, subsample=0.9 
[CV]  criteri

[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:   13.8s finished


0.8340142077678256
{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.5, 'n_estimators': 10, 'subsample': 0.9}


### Set de entrenamiento

In [31]:
t0 = time()


# Parámetros óptimos
params_optimos_gb = {'criterion': 'friedman_mse','learning_rate':0.2, 'n_estimators':10, 'max_depth':5,\
                     'min_samples_split':0.5,'min_samples_leaf':0.1, 'subsample':0.9, 'max_features':'sqrt',\
                     } #'loss': 'deviance'

# Creamos el gradient boosting
gra_boost = GradientBoostingRegressor(**params_optimos_gb)

# Lo entrenamos con nuestro set de entrenamiento
gra_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = gra_boost.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.8168973775511638
Tiempo de ejecución: 0.03341 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingRegressor(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = gra_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingRegressor(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = gra_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

#### Submit con probabilidades

# Logistic Regression

### Grid Search

In [27]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con nuestro set de entrenamiento
log_reg.fit(x_train,y_train)
params = {
    'penalty':['l1', 'l2'],        # l1 is Lasso, l2 is Ridge
    'solver':['liblinear'],
    'C': np.linspace(0.00002,1,100)
}

lr = LogisticRegression()
lr_gs = GridSearchCV(lr, params, cv=3, verbose=1).fit(x_train, y_train)

print ("Best Params", lr_gs.best_params_)
print ("Best Score", lr_gs.best_score_)


tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Fitting 3 folds for each of 200 candidates, totalling 600 fits
Best Params {'C': 0.01012080808080808, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score 0.9493818681318681
Tiempo de ejecución: 316.94204 seconds.


[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  5.3min finished


### Set de entrenamiento

In [35]:
t0 = time()

best_params = {\
    'penalty':'l2',\
    'solver':'liblinear',\
    'C': 0.01012080808080808\
}
lr_best = LogisticRegression(**best_params)
lr_best.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = lr_best.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5058165366276549
Tiempo de ejecución: 0.11796 seconds.


### Test final

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = log_reg.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con probabilidades

# Decision Tree

### Grid Search

In [71]:
t0 = time()

g_cv = GridSearchCV(DecisionTreeRegressor(random_state=0),
              param_grid={'min_samples_split': np.arange(2, 10)},
              scoring='roc_auc', cv=5, refit=True)
g_cv.fit(x,y)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)
print(g_cv.best_params_)

Tiempo de ejecución: 24.13923 seconds.
{'min_samples_split': 8}


### Set de entrenamiento

In [70]:

t0 = time()

# Parámetros óptimos
params_optimos_dt = {'criterion':"mse",'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}

# Creamos el logistic regression
decision_tree = DecisionTreeRegressor(**params_optimos_dt)

# Lo entrenamos con nuestro set de entrenamiento
decision_tree.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = decision_tree.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5916255779511632
Tiempo de ejecución: 0.20824 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = decision_tree.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = decision_tree.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_decision_tree_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# Bagging

## 1 Con Logistic Regression

### Set de entrenamiento

In [80]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingRegressor(**params_optimos_bag_lr)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5074632459182199
Tiempo de ejecución: 9.35253 seconds.


### Test final

#### Predicciones con valores binarios

In [79]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Tiempo de ejecución: 11.51247 seconds.


#### Predicciones con probabilidades

In [ ]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

#### Submit con probabilidades

## 2 Con Random Forest

### Grid Search

In [85]:
params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

clf = GridSearchCV(BaggingRegressor(RandomForestRegressor(**params_optimos_rf),
                                     n_estimators = 100, max_features = 0.5),
                   param_grid, scoring = 'roc_auc')

clf.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=BaggingRegressor(base_estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_..._estimators=100, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'base_estimator__max_depth': [1, 2, 3, 4, 5], 'max_samples': [0.05, 0.1, 0.2, 0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [86]:
clf.best_estimator_.fit(x_train,y_train)
y_pred = clf.best_estimator_.predict(x_test)
roc_auc_score(y_test,y_pred)

0.844373103597746

### Set de entrenamiento

In [82]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_dt = {'base_estimator':RandomForestRegressor(**params_optimos_rf), 'n_estimators':10, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging


# Parámetros óptimos
params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}


bagging = BaggingRegressor(**params_optimos_bag_dt)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.8473703222077735
Tiempo de ejecución: 3.98110 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

#### Submit con probabilidades

# Voting

### Set de entrenamiento

In [32]:
# Creamos los estimadores base que utilizaremos para el voting
estimador_1 = DecisionTreeRegressor(**params_optimos_dt)
estimador_2 = GradientBoostingRegressor(**params_optimos_gb)
estimador_3 = ExtraTreesRegressor(**params_optimos_et)

# Los agregamos a la lista de estimadores
estimadores = []
estimadores.append(('Naive Bayes', estimador_1))
estimadores.append(('Logistic Regression', estimador_2))
estimadores.append(('Random Forest', estimador_3))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5014331382748157
Tiempo de ejecución: 3.12942 seconds.


/home/mariano/ENTER/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Voting NN y XGBRegressor

In [53]:
# Initialising the NN
model = Sequential()

# layers
model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = len(features)))
model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

estimador_1 = model
estimador_2 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)

estimadores = []
estimadores.append(('NN', estimador_1))
estimadores.append(('XGBRegressor', estimador_2))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)


TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x7f52e0766d30>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con la totalidad del set de datos
voting.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = voting.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

# ENSEMBLE

In [54]:
xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)

params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

random_forest = RandomForestRegressor(**params_optimos_rf)

knn = KNeighborsRegressor(n_neighbors=65)

params_optimos_gb = {'criterion': 'friedman_mse','learning_rate':0.2, 'n_estimators':10, 'max_depth':5,\
                     'min_samples_split':0.5,'min_samples_leaf':0.1, 'subsample':0.9, 'max_features':'sqrt',\
                     } #'loss': 'deviance'

gra_boost = GradientBoostingRegressor(**params_optimos_gb)

params_optimos_ab = {'n_estimators':50}


ada_boost = AdaBoostRegressor(**params_optimos_ab)

In [55]:
from sklearn.cross_validation import KFold
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [56]:
t0 = time()
ada_oof_train, ada_oof_test = get_oof(ada_boost, x, y, x_test_final) # Ada
rf_oof_train, rf_oof_test = get_oof(random_forest,x, y, x_test_final) # Random Forest
knn_oof_train, knn_oof_test = get_oof(knn, x, y, x_test_final) # KNN 
gb_oof_train, gb_oof_test = get_oof(gra_boost,x, y, x_test_final) # Gradient Boost
xgb_oof_train, xgb_oof_test = get_oof(xgb, x, y, x_test_final) # XGB
tf = (time() - t0)/60
print("Tiempo de ejecución: %0.5f minutes." % tf)

Tiempo de ejecución: 4.60493 minutes.


In [57]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'Ada': ada_oof_train.ravel(),
     'KNN': knn_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel(),
        'Xgb' : xgb_oof_train.ravel()
    })
base_predictions_train.head()

,RandomForest,Ada,KNN,GradientBoost,Xgb
0,0.033605,0.008776,0.061538,0.041120,0.024746
1,0.103272,0.169923,0.000000,0.083632,0.108489
2,0.059701,0.008776,0.138462,0.083689,0.014804
3,0.157759,0.169923,0.076923,0.106277,0.211018
4,0.014613,0.008776,0.000000,0.017356,0.008419


In [58]:
import plotly.graph_objs as go
import plotly.offline as py

data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Portland',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

In [59]:
data

[Heatmap({
     'colorscale': 'Portland',
     'reversescale': True,
     'showscale': True,
     'x': array(['RandomForest', 'Ada', 'KNN', 'GradientBoost', 'Xgb'], dtype=object),
     'y': array(['RandomForest', 'Ada', 'KNN', 'GradientBoost', 'Xgb'], dtype=object),
     'z': array([[1.        , 0.73154968, 0.71543037, 0.83689299, 0.92413388],
                 [0.73154968, 1.        , 0.51111199, 0.61529702, 0.7604925 ],
                 [0.71543037, 0.51111199, 1.        , 0.74349365, 0.62963415],
                 [0.83689299, 0.61529702, 0.74349365, 1.        , 0.77924601],
                 [0.92413388, 0.7604925 , 0.62963415, 0.77924601, 1.        ]])
 })]

In [60]:
x_train = np.concatenate(( ada_oof_train, rf_oof_train, knn_oof_train, gb_oof_train,xgb_oof_train), axis=1)
x_test = np.concatenate(( ada_oof_test, rf_oof_test, knn_oof_test, gb_oof_test,xgb_oof_test), axis=1)

In [61]:
X = x_train
y = y
y_mean = np.mean(y)

In [62]:
from sklearn.model_selection import train_test_split
import xgboost as xgb


Xtr, Xv, ytr, yv = train_test_split(X, y, test_size=0.2, random_state=2000)

dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

xgb_params = {
    'booster':'gbtree',
    'colsample_bylevel':1,
    'colsample_bytree':0.6,
    'gamma':5,
    'learning_rate':0.02,
    'max_delta_step':0,
    'min_child_weight':5,
    'missing':None,
    'n_estimators':600,
    'objective':'binary:logistic',
    'max_depth': 5,
    'subsample': 1.0,
    'scale_pos_weight': 1,
    'random_state':0,
    'reg_lambda':1,
    'reg_alpha':0,
    'nthread': 1,    
    'n_jobs': 1,
    'silent': True,
    'base_score':0.5,
}

In [63]:
model_xgb = xgb.train(xgb_params, dtrain, 2000, watchlist, early_stopping_rounds=5000,
                  maximize=False, verbose_eval=15)

[0]	train-error:0.049063	valid-error:0.056142
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 5000 rounds.
[15]	train-error:0.047518	valid-error:0.05537
[30]	train-error:0.047518	valid-error:0.05537
[45]	train-error:0.046166	valid-error:0.055112
[60]	train-error:0.045908	valid-error:0.053567
[75]	train-error:0.045908	valid-error:0.053309
[90]	train-error:0.045973	valid-error:0.053052
[105]	train-error:0.045973	valid-error:0.053309
[120]	train-error:0.045779	valid-error:0.053052
[135]	train-error:0.045651	valid-error:0.052537
[150]	train-error:0.045522	valid-error:0.052537
[165]	train-error:0.045586	valid-error:0.052794
[180]	train-error:0.045586	valid-error:0.052794
[195]	train-error:0.045586	valid-error:0.052794
[210]	train-error:0.045522	valid-error:0.052794
[225]	train-error:0.045522	valid-error:0.052794
[240]	train-error:0.045457	valid-error:0.052794
[255]	train-error:0.045457	valid-error:0.0527

In [64]:
dtest = xgb.DMatrix(x_test)
predicted_test_xgb = model_xgb.predict(dtest)

In [65]:
def border_submit(x):
    if x>1:
        return 1
    elif x<0:
        return 0
    else: return x

In [66]:
person = np.array(test['person'])
person =person.reshape((19415,1))
submit = pd.DataFrame(data=np.column_stack((person,predicted_test_xgb)),columns=['person','label'])
submit['label'] = submit['label'].apply(lambda x: border_submit(x))
submit.to_csv('submit_ensemble.csv', index=False)